# Morfología Matemática

En este notebook, vamos a mostrar los elementos y operadores morfológicos esenciales dentro del procesamiento de imágenes.

El alumno puede hacer uso de cualquier herramienta software, aunque se recomienda el uso de sci-kit image (tal y como lo vamos a mostrar en este notebook)

In [ ]:
# Para realizar operaciones sobre matrices
import numpy as np
# Para procesar las imágenes
from skimage import data
# Para mostrar las imágenes
import matplotlib.pyplot as plt

In [ ]:
# Paquetes necesarios para la conversión de imágenes de color a escala de grises
from skimage.color import rgb2gray

In [ ]:
# Paquetes necesarios para la morfología matemática
from skimage.morphology import erosion, dilation, opening, closing
# Elementos estructurales
from skimage.morphology import disk, diamond, ball, rectangle

In [ ]:
# Defino una función para mostrar una imagen por pantalla con el criterio que considero más acertado
def imshow(img):
    fig, ax = plt.subplots(figsize=(7, 7))
    # El comando que realmente muestra la imagen
    ax.imshow(img,cmap=plt.cm.gray)
    # Para evitar que aparezcan los números en los ejes
    ax.set_xticks([]), ax.set_yticks([])
    plt.show()

In [ ]:
def plot_comparison(original, filtered, filter_name):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(20, 10), sharex=True,
                                   sharey=True)
    ax1.imshow(original, cmap=plt.cm.gray)
    ax1.set_title('Imagen Original')
    ax1.axis('off')
    ax1.set_adjustable('box')
    ax2.imshow(filtered, cmap=plt.cm.gray)
    ax2.set_title(filter_name)
    ax2.axis('off')
    ax2.set_adjustable('box')
    plt.show()

In [ ]:
# Leo la imagen chelsea (es un gato)
chelsea = data.chelsea()

imshow(chelsea)

# Elementos estructurales

In [ ]:
# Elementos tipo disco (los más empleados)
# Especificamos el radio del disco
elemento_estructural_disco = disk(20)
# Lo mostramos por pantalla
elemento_estructural_disco

In [ ]:
imshow(elemento_estructural_disco)

In [ ]:
# Elementos tipo diamante (los más empleados)
# Especificamos el radio del diamante
elemento_estructural_diamante = diamond(30)
# Lo mostramos por pantalla
elemento_estructural_diamante

In [ ]:
imshow(elemento_estructural_diamante)

In [ ]:
# Si ahora comparamos dos elementos estructurales (disco y diamante),
# vemos su diferencia cuando los valores son muy grandes
imshow(np.abs(diamond(5)-disk(5)))

# Erosión

In [ ]:
elemento_estructural_disco = disk(10)
chelsea_gray = rgb2gray(chelsea)
chelsea_eroded = erosion(chelsea_gray,elemento_estructural_disco)

In [ ]:
plot_comparison(chelsea_gray,chelsea_eroded,'Erosión')

In [ ]:
plot_comparison(chelsea_gray,erosion(chelsea_gray,disk(20)),'Erosión')

In [ ]:
plot_comparison(chelsea_gray,erosion(chelsea_gray,disk(2)),'Erosión')

In [ ]:
# Obtenemos la diferencia entre la imagen original y la imagen erosionada
imshow(chelsea_gray-erosion(chelsea_gray,disk(2)))

In [ ]:
# Para obtener el histograma de una imagen (en este caso nbins 10)
from skimage import data, exposure, img_as_float
exposure.histogram(chelsea_gray, nbins=10)

In [ ]:
# Nos quedamos con esta componente (la llamamos red porque luego la pintaremos en la capa roja de color)
red = chelsea_gray-erosion(chelsea_gray,disk(2))>0.2

In [ ]:
# Hacemos una copia de la imagen original
chelsea_color = chelsea.copy()

In [ ]:
# Primero binarizamos la imagen, y posteriormente, la multiplicamos por 255 (que es el color blanco)
chelsea_color[:,:,0] = 255*(red>0)

In [ ]:
imshow(chelsea_color)

In [ ]:
# Tiempo de procesamiento de estos algoritmos
import time
arrayRadius = [1,2,5,10,25,40,50,75,100]
diffTime = []
for radius in arrayRadius:
    print('radius '+str(radius))
    tic = time.perf_counter()
    chelsea_eroded = erosion(chelsea_gray,disk(radius))
    toc = time.perf_counter()
    diffTime.extend([toc-tic])

In [ ]:
# Tiempo de procesamiento de estos algoritmos
import time
arrayRadius = [1,2,5,10,25,40,50,75,100]
diffTimeOpening = []
for radius in arrayRadius:
    print('radius '+str(radius))
    tic = time.perf_counter()
    chelsea_opening = opening(chelsea_gray,disk(radius))
    toc = time.perf_counter()
    diffTimeOpening.extend([toc-tic])

In [ ]:
plt.plot(arrayRadius, diffTime,"g-",label='Erosion')
plt.plot(arrayRadius, diffTimeOpening, "b--",label='Opening')
plt.xlabel('Tamaño del radio')
plt.ylabel('Tiempo (segundos)')
plt.legend()
plt.show()

In [ ]:
# Mediana de las diferencias entre erosión y opening
np.median([diffTimeOpening[i]/diffTime[i] for i in range(len(diffTime))])

# Para hacer ejercicios de segmentación

In [ ]:
from skimage.segmentation import (active_contour, felzenszwalb, quickshift,
                                 mark_boundaries, slic, clear_border)

In [ ]:
coffeeSegmented = slic(data.coffee(),n_segments=50)

In [ ]:
randomImage = ((coffeeSegmented == 11) | (coffeeSegmented == 8) | (coffeeSegmented == 22) | (coffeeSegmented == 45))

In [ ]:
imshow(randomImage)

In [ ]:
imshow(opening(randomImage,disk(10)))

In [ ]:
imshow(closing(randomImage,disk(10)))

### ¿Cómo se relacionan la erosión y la dilatación?

In [ ]:
closingVersion2 = (1-closing(1-randomImage,disk(10)))

In [ ]:
openingVersion1 = opening(randomImage,disk(10))

In [ ]:
imshow(openingVersion1)

In [ ]:
imshow(1-randomImage)

In [ ]:
imshow(closing(1-randomImage,disk(10)))

In [ ]:
imshow(closingVersion2)

In [ ]:
np.sum(closingVersion2-openingVersion1)